In [1]:
!pip3 install seaborn bs4 textblob lxml tflearn


    100% |################################| 215kB 3.3MB/s ta 0:00:01
    100% |################################| 645kB 1.8MB/s ta 0:00:01
    100% |################################| 5.7MB 259kB/s ta 0:00:011
    100% |################################| 102kB 6.0MB/s a 0:00:011
    100% |################################| 102kB 5.5MB/s ta 0:00:01
  Running setup.py bdist_wheel for bs4 ... done
  Stored in directory: /root/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
  Running setup.py bdist_wheel for tflearn ... done
  Stored in directory: /root/.cache/pip/wheels/d0/f6/69/0ef3ee395aac2e5d15d89efd29a9a216f3c27767b43b72c006
Successfully built bs4 tflearn


In [2]:
#https://tm3.ghost.io/2017/04/19/amazon-food-reviews-part-i/
#https://tm3.ghost.io/2017/04/21/amazon-food-reviews-part-vi/

from __future__ import division
from __future__ import print_function
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pylab
from bs4 import BeautifulSoup
import textblob
from textblob import TextBlob
import pickle
import datetime
from scipy.stats import kendalltau
data = pd.read_csv("./Review100.csv")
print(data.dtypes)


FileNotFoundError: [Errno 2] File b'./Review100.csv' does not exist: b'./Review100.csv'

In [ ]:
#clean text using bs4 - approx. 10 minutes
data['text_cln']= data['Text'].map(lambda x: BeautifulSoup(x, "html.parser").get_text())

#apply the polarity score to each text feature using 
#TextBlob ()

data['tb_polarity']= data['text_cln'].map(lambda x: 
TextBlob(x).sentiment.polarity)

#pickle
data.to_pickle('amz_data.pkl')




In [ ]:
data = pd.read_pickle('amz_data.pkl')
print(data)

In [ ]:
import time
from time import time
import keras
from keras.datasets import reuters
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import LSTM
from keras.preprocessing.text import Tokenizer
import tensorflow as tf
from keras.models import load_model

#Read in data
#read pickle from previous blog post Part I

data = pd.read_pickle('amz_data.pkl')

#normalize date time
data2 = data.copy()
data2['datetime'] = data2['Time'].map(lambda x: (datetime.datetime.fromtimestamp(int(x)).strftime('%Y-%m-%d %H:%M:%S')))
data2['datetime'] = pd.to_datetime(data2['datetime'])


In [ ]:
#train/test split 80/20
from sklearn.model_selection import train_test_split
X_data, y_target = data2['text_cln'], data2['Score']
y_target = y_target.values
X_datatrain, X_datatest, y_train, y_test = train_test_split(X_data, y_target, test_size=0.2, random_state=0)


In [ ]:
# Process vocabulary  
import time
import tflearn

MAX_DOCUMENT_LENGTH = 800
vocab_processor = tflearn.data_utils.VocabularyProcessor(MAX_DOCUMENT_LENGTH)
start_time = time.time() #timing it 

x_train = np.array(list(vocab_processor.fit_transform(X_datatrain))) 
print("--- %s seconds ---" % (time.time() - start_time))  

x_test = np.array(list(vocab_processor.transform(X_datatest)))
print("--- %s seconds ---" % (time.time() - start_time))  

n_words = len(vocab_processor.vocabulary_)
print("--- %s seconds ---" % (time.time() - start_time))  

print('Total words: %d' % n_words)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

#confirm that the shape is consistent with max_document_lenght = 800
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

In [ ]:
max_words = MAX_DOCUMENT_LENGTH
batch_size = 32
#An epoch is a full pass over your training data
#this one will pass over the training set 5 times
#In 32 batches
epochs = 500

In [ ]:
#number of target classes

num_classes = int(np.max(y_train))+1
print(num_classes, 'classes')
'''
print('Convert class vector to binary class matrix '
  '(for use with categorical_crossentropy)')
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

'''
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)
print(y_test)

In [ ]:
print('Building model...')
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(512, input_shape=(max_words,)))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(num_classes))
model.add(tf.keras.layers.Activation('softmax'))

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print("--- %s seconds ---" % (time.time() - start_time))  

#timed it at around 8 minutes
history = model.fit(x_train, y_train,
                batch_size=batch_size,
                epochs=epochs,
                verbose=1,
                validation_split=0.1)
print("--- %s seconds ---" % (time.time() - start_time)) 

In [ ]:
score = model.evaluate(x_test, y_test,
                   batch_size=batch_size, verbose=1)
print("--- %s seconds ---" % (time.time() - start_time))  

print('Test score:', score[0])
print('Test accuracy:', score[1])

In [ ]:
model.predict_classes(x_test, batch_size=batch_size, verbose=1)
model.predict_proba(x_test, batch_size=batch_size, verbose=1)